In [ ]:
import openai
import pandas as pd
import numpy as np
import os

def generate_queries(theme, context, num_queries=3, model="text-davinci-003", temperature=0.7):
    """
    Generates search queries based on a theme and historical context using the OpenAI API.
    
    :param theme: The theme or research question to base queries on.
    :param context: Historical text to provide context for the query generation.
    :param num_queries: Number of queries to generate.
    :param model: The OpenAI GPT model to use.
    :param temperature: The temperature to use for the generation (controls randomness).
    :return: A list of generated queries.
    """
    # Constructing the prompt
    #prompt = f"Givet en övergripande forskningsfråga '{theme}' och den följande texten: '{context}', skapa {num_queries} eller färre frågor som relaterar till den övergripande forskningsfrågan, och som har sitt svar i texten. Om inga relaterade frågor finns i texten, så returnera inga frågor:"
    
    openai.api_key = os.getenv('OPENAI_KEY_WORKING')
    response = openai.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": f"Vänligen använd {theme} som min övergripande sökfråga och generera nya frågor baserade på denna text: {context}. Jag önskar att dessa nya frågor relaterar till: {theme} och att du också anger var i: {context} de är hämtade från för att ge mig referenser. Motivera också kopplingen mellan frågan och referensen."}
        #{"role": "assistant", "content": f"Här är texten som ska innehålla svaret på de mer detaljerade frågorna eller sökningarna: {context}"}
        #{"role": "user", "content": "Har de extraherade frågorna sitt svar i kontexten, och är de relaterade till den övergripande frågeställningen: {theme}"}
    ]
    
    )
    return response

In [ ]:
import json

with open('/workspaces/huminfra_hackathon/semantic_search/reports.json', 'r') as f:
    volumes = json.load(f)
with open('/workspaces/huminfra_hackathon/semantic_search/output/example_questions.txt', 'w') as f:
    for vol_key, value in volumes.items():
        for i, report in enumerate(value['reports']):
            chunk = '\n'.join(report)
            answer = generate_queries(
                theme='Hur arbetade polisen i Göteborg under 1800-talets senare hälft? (arbetsuppgifter, fördelning av arbetsuppgifter, gjorde konstaplarna samma saker osv)',
                context=chunk,
                num_queries=3
            )
            f.write(answer.choices[0].message.content)
            f.write(f"volume: {vol_key}\nreport: {i}\ntext: {chunk}\n***\n")
            print(i)
        break